# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 

In [2]:
spark = SparkSession \
.builder \
.appName('wrangling quiz') \
.getOrCreate()

In [3]:
log = spark.read.json('data/sparkify_log_small.json')

# Question 1

Which page did user id "" (empty string) NOT visit?

In [4]:
# TODO: write your code to answer question 1
log.select(['userId', 'page']).where(log.userId == "").dropDuplicates().sort("page").show()

+------+-----+
|userId| page|
+------+-----+
|      |About|
|      | Help|
|      | Home|
|      |Login|
+------+-----+



# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [17]:
log.where(log.userId == '').take(4)

[Row(artist=None, auth='Logged Out', firstName=None, gender=None, itemInSession=0, lastName=None, length=None, level='free', location=None, method='PUT', page='Login', registration=None, sessionId=5598, song=None, status=307, ts=1513721196284, userAgent=None, userId=''),
 Row(artist=None, auth='Logged Out', firstName=None, gender=None, itemInSession=26, lastName=None, length=None, level='paid', location=None, method='GET', page='Home', registration=None, sessionId=428, song=None, status=200, ts=1513721274284, userAgent=None, userId=''),
 Row(artist=None, auth='Logged Out', firstName=None, gender=None, itemInSession=5, lastName=None, length=None, level='free', location=None, method='GET', page='Home', registration=None, sessionId=2941, song=None, status=200, ts=1513722009284, userAgent=None, userId=''),
 Row(artist=None, auth='Logged Out', firstName=None, gender=None, itemInSession=5, lastName=None, length=None, level='paid', location=None, method='GET', page='Home', registration=None, 

In [15]:
log.where(log.userId == '').where(log.level =='paid').take(4)

[Row(artist=None, auth='Logged Out', firstName=None, gender=None, itemInSession=26, lastName=None, length=None, level='paid', location=None, method='GET', page='Home', registration=None, sessionId=428, song=None, status=200, ts=1513721274284, userAgent=None, userId=''),
 Row(artist=None, auth='Logged Out', firstName=None, gender=None, itemInSession=5, lastName=None, length=None, level='paid', location=None, method='GET', page='Home', registration=None, sessionId=5243, song=None, status=200, ts=1513723183284, userAgent=None, userId=''),
 Row(artist=None, auth='Logged Out', firstName=None, gender=None, itemInSession=6, lastName=None, length=None, level='paid', location=None, method='PUT', page='Login', registration=None, sessionId=5243, song=None, status=307, ts=1513723184284, userAgent=None, userId=''),
 Row(artist=None, auth='Logged Out', firstName=None, gender=None, itemInSession=14, lastName=None, length=None, level='paid', location=None, method='GET', page='Home', registration=None,

# Question 3

How many female users do we have in the data set?

In [18]:
# TODO: write your code to answer question 3

log.select(['userId','gender']).dropDuplicates().sort("userId").groupby(log.gender).count().collect()

[Row(gender='F', count=462),
 Row(gender=None, count=1),
 Row(gender='M', count=501)]

# Question 4

How many songs were played from the most played artist?

In [19]:
log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [34]:
# TODO: write your code to answer question 4
new_df = log.groupBy('artist').count()

In [43]:
new_df.sort('count', ascending=False).collect()

[Row(artist=None, count=1653),
 Row(artist='Coldplay', count=83),
 Row(artist='Kings Of Leon', count=69),
 Row(artist='Florence + The Machine', count=52),
 Row(artist='BjÃ\x83Â¶rk', count=46),
 Row(artist='Dwight Yoakam', count=45),
 Row(artist='Justin Bieber', count=43),
 Row(artist='The Black Keys', count=40),
 Row(artist='OneRepublic', count=37),
 Row(artist='Jack Johnson', count=36),
 Row(artist='Muse', count=36),
 Row(artist='Radiohead', count=31),
 Row(artist='Taylor Swift', count=29),
 Row(artist='Lily Allen', count=28),
 Row(artist='Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', count=28),
 Row(artist='Train', count=28),
 Row(artist='Nickelback', count=27),
 Row(artist='Metallica', count=27),
 Row(artist='Daft Punk', count=27),
 Row(artist='Kanye West', count=26),
 Row(artist='John Mayer', count=24),
 Row(artist='Red Hot Chili Peppers', count=24),
 Row(artist='Alliance Ethnik', count=24),
 Row(artist='Linkin Park', count=23),
 Row(artist='Eminem', coun

# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [58]:
# TODO: write your code to answer question 5
function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())
user_window = Window \
    .partitionBy('userID') \
    .orderBy(desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

In [ ]:
user_window.show()

In [59]:
cusum = log.filter((log.page == 'NextSong') | (log.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(col('page'))) \
    .withColumn('period', Fsum('homevisit').over(user_window))

In [60]:
cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+

